# `ProteinWorkshop` Tutorial, Part 1 - Training a New Model
![Models](../docs/source/_static/box_models.png)

## Train a new model using the `ProteinWorkshop`

In [14]:
%load_ext autoreload
%autoreload 2
# %load_ext blackcellmagic

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Welcome to the tutorial series for the `ProteinWorkshop`! 

In the `ProteinWorkshop`, we implement numerous [featurisation](https://www.proteins.sh/configs/features) schemes, [datasets](https://www.proteins.sh/configs/dataset) for [self-supervised pre-training](https://proteins.sh/quickstart_component/pretrain.html) and [downstream evaluation](https://proteins.sh/quickstart_component/downstream.html), [pre-training](https://proteins.sh/configs/task) tasks, and [auxiliary tasks](https://proteins.sh/configs/task.html#auxiliary-tasks).

[Processed datasets](https://drive.google.com/drive/folders/18i8rLST6ZICTBu6Q67ClT0KqN9AHeqoW?usp=sharing) and [pre-trained weights](https://drive.google.com/drive/folders/1zK1r8FpmGaqV_QwUJuvDacwSL0RW-Vw9?usp=sharing) are made available. Downloading datasets is not required; upon first run all datasets will be downloaded and processed from their respective source.

The `ProteinWorkshop` encompasses several models as well as pre-trained weights for them so that you can readily use them.

In this tutorial, we show you how you can use what is already available in the protein workshop to train and use models for specific tasks. The `ProteinWorkshop` is structured as a very modular package; we will therefore talk about how to change the different parts of it, like the model, training task, dataset, featurization scheme, etc. in this tutorial. 

Besides using all the different options we provide, you can make use of the modular nature of the `ProteinWorkshop` to add your own models, datasets, featurization schemes, and training tasks. We will show you how to do this in the next tutorials.

To train a new model, you can follow the following 3-step procedure:

1. Choose the parts you want to consider: model, training task, dataset, featurization scheme and auxiliary tasks
2. Validate the designed training config
3. Use the designed config to train a new model

### 1. Choose the parts you want to consider: model, training task, dataset, featurization scheme and auxiliary tasks

You can switch out any of these for another available option by replacing the corresponding argument's value in `overrides`:

`cfg = hydra.compose("template", overrides=["encoder=schnet", "task=inverse_folding", "dataset=afdb_swissprot_v4", "features=ca_base", "+aux_task=none"], return_hydra_config=True)`

In [15]:
# Misc. tools
import os

# Hydra tools
import hydra

from hydra.compose import GlobalHydra
from hydra.core.hydra_config import HydraConfig

from proteinworkshop.constants import HYDRA_CONFIG_PATH
from proteinworkshop.utils.notebook import init_hydra_singleton

version_base = "1.2"  # Note: Need to update whenever Hydra is upgraded
init_hydra_singleton(reload=True, version_base=version_base)

path = HYDRA_CONFIG_PATH
rel_path = os.path.relpath(path, start=".")
# print(rel_path)
GlobalHydra.instance().clear()
hydra.initialize(rel_path, version_base=version_base)

cfg = hydra.compose(
    config_name="train",
    overrides=[
        "encoder=schnet",
        # "decoder.graph_label.dummy=True",
        "task=multiclass_graph_classification",
        "dataset=ec_reaction",
        # "features=ca_base", 
        # "+aux_task=none",
        # "trainer.max_epochs=1000",
        # "encoder.num_blocks=4",
        # "encoder.dropout=0.3",
        # "encoder.out_channels=128",
        # "optimiser.optimizer.lr=2e-4",
        # "callbacks.early_stopping.patience=10",
        # "test=True",
        # "trainer.devices=[1]",
        # "optimizer.weight_decay=0.5"
    ],
    return_hydra_config=True,
)

# Note: Customize as needed e.g., when running a sweep
cfg.hydra.job.num = 0
cfg.hydra.job.id = 0
cfg.hydra.hydra_help.hydra_help = False
cfg.hydra.runtime.output_dir = "outputs"

HydraConfig.instance().set_config(cfg)

[07/10/24 01:19:14] INFO     Hydra singleton cleared and ready to re-initialise.                     ]8;id=682590;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/utils/notebook.py\notebook.py]8;;\:]8;id=446580;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/utils/notebook.py#66\66]8;;\

                    INFO     Hydra initialised at                                                    ]8;id=595341;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/utils/notebook.py\notebook.py]8;;\:]8;id=321054;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/utils/notebook.py#53\53]8;;\
                             /home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/config.                       

### 2. Validate the designed training config

This is not strictly necessary, but it is a good idea to validate the config before training. This will check that all the arguments you have provided are valid and that the config is complete.

In [16]:
from proteinworkshop.configs import config

cfg = config.validate_config(cfg)

[07/10/24 01:19:15] DEBUG    CUDA available: True                                                     ]8;id=123257;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=808210;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py#249\249]8;;\

                    DEBUG    Requested GPUs: 1.                                                       ]8;id=403619;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=824081;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py#250\250]8;;\

                    DEBUG    GPU count set to: 1                                                      ]8;id=164173;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=667350;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py#255\255]8;;\

                    WARNING  You are not using early stopping.                                        ]8;id=776511;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py\config.py]8;;\:]8;id=287828;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/configs/config.py#165\165]8;;\

In [17]:
print(cfg.keys())
for key in cfg.keys():
    print(key)
    print(cfg[key])

dict_keys(['hydra', 'env', 'dataset', 'features', 'encoder', 'decoder', 'transforms', 'callbacks', 'optimiser', 'scheduler', 'trainer', 'extras', 'metrics', 'task', 'logger', 'hparams', 'name', 'seed', 'num_workers', 'task_name', 'test'])
hydra
{'run': {'dir': '${env.paths.log_dir}/${task_name}/runs/${now:%Y-%m-%d}_${now:%H-%M-%S}'}, 'sweep': {'dir': '${env.paths.log_dir}/${task_name}/multiruns/${now:%Y-%m-%d}_${now:%H-%M-%S}', 'subdir': '${hydra.job.num}'}, 'launcher': {'_target_': 'hydra._internal.core_plugins.basic_launcher.BasicLauncher'}, 'sweeper': {'_target_': 'hydra._internal.core_plugins.basic_sweeper.BasicSweeper', 'max_batch_size': None, 'params': None}, 'help': {'app_name': '${hydra.job.name}', 'header': '${hydra.help.app_name} is powered by Hydra.\n', 'footer': 'Powered by Hydra (https://hydra.cc)\nUse --hydra-help to view Hydra specific help\n', 'template': '${hydra.help.header}\n== Configuration groups ==\nCompose your configuration from those groups (group=option)\n\n$A

### 3. Use the designed config to train a new model

Now with the config you have designed, you can train a new model. You can also use the `ProteinWorkshop` to evaluate the model on a downstream task.

In [18]:
from proteinworkshop.train import train_model

train_model(cfg)

Seed set to 52


                    INFO     Instantiating datamodule:                                                  ]8;id=537486;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/train.py\train.py]8;;\:]8;id=507365;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/train.py#99\99]8;;\
                             <proteinworkshop.datasets.ec_reaction.EnzymeCommissionReactionDataset...              

                    INFO     Setting up EC Prediction dataset. Fraction 1.0                       ]8;id=35708;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/datasets/ec_reaction.py\ec_reaction.py]8;;\:]8;id=963896;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/datasets/ec_reaction.py#55\55]8;;\

                    INFO     Instantiating callbacks...                                                ]8;id=870905;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/train.py\train.py]8;;\:]8;id=192045;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/train.py#106\106]8;;\

                    INFO     Instantiating callback <lightning.pytorch.callbacks.ModelCheckpoint>   ]8;id=35855;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/utils/callbacks.py\callbacks.py]8;;\:]8;id=570567;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/utils/callbacks.py#31\31]8;;\

...

                    INFO     All structures already processed and overwrite=False. Skipping download.   ]8;id=76873;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/datasets/base.py\base.py]8;;\:]8;id=548135;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/datasets/base.py#327\327]8;;\

                    INFO     All structures already processed and overwrite=False. Skipping download.   ]8;id=362065;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/datasets/base.py\base.py]8;;\:]8;id=600310;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/datasets/base.py#362\362]8;;\

                    INFO     Reading data into memory                                                   ]8;id=13065;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/datasets/base.py\base.py]8;;\:]8;id=583288;file:///home/zhang/Projects/3d/ProteinWorkshop/proteinworkshop/datasets/base.py#337\337]8;;\

0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [00:00<?, ?it/s]
  0%|          | 0/2558 [

RecursionError: maximum recursion depth exceeded in comparison

### 4. Wrapping up

Have any additional questions about using the components provided in the `ProteinWorkshop`? [Create a new issue](https://github.com/a-r-j/ProteinWorkshop/issues/new/choose) on our [GitHub repository](https://github.com/a-r-j/ProteinWorkshop). We would be happy to work with you to leverage the full power of the repository!

/home/yang/anaconda3/envs/3d/lib/python3.10/site-packages/lightning/pytorch/utilities/data.py